In [0]:
import torch
import numpy as np
import random
from scipy.special import softmax

In [0]:
def gen_attack(N, x_orig, t, delta_max, alpha, rho, tau, G, model, device):
    # Algo 1 from : https://arxiv.org/pdf/1805.11090.pdf
    #Convergence - best fitness difference within episolon for n generations 
    #
    """
        N : size of population
        x_orig: original example (batch, channel, x, y), batch = 1
        t: target label
        delta_max: maxmimum distance 
        alpha: mutation range 
        rho: mutation probability (~ 1% - 3%)
        tau: sampling temperature
        G: # of generations
        model: the attacked model
        device: hardware the tensors will be stored on
    """
    # initialize population
    dims = list(x_orig.size())
    dims[0] = N
    # population is an (N, 1, 28, 28) in the case of MNIST
    population = torch.empty(dims, device=device).uniform_(-delta_max, delta_max)
    population = torch.clamp(population + x_orig, 0, 1) - x_orig

    # for g in range(G):
    #     for i in population:
    #         # compute fitness
    pass    

In [0]:
def mutation_op(cur_pop, x_orig, idx, alpha=0.15, rho=0.1, delta_max=0.1):
    """
        cur_pop: the current population
        x_orig :  the image we are using for the attack
        idx    : an index. useful for debugging
        alpha: mutation range
        rho: mutation probability
        delta_max: maxmimum distance
    """
    step_noise = alpha * delta_max
    perturb_noise = torch.empty_like(cur_pop).uniform_(-step_noise, step_noise)
    mask = torch.empty_like(cur_pop).bernoulli_(rho)
    mutated_pop = perturb_noise * mask + cur_pop
    clamped_mutation_pop = torch.clamp(mutated_pop + x_orig, 0, 1) - x_orig
    normalized_pop = torch.clamp(clamped_mutation_pop, -delta_max, delta_max)
    return normalized_pop

# x = torch.FloatTensor(2, 2)
# image = torch.FloatTensor(2, 2).uniform_(0, 1)
# print(x, image)
# mutpop = mutation_op(x, image, 1)

In [0]:
def fitness(model, batch, target_class):
    """
        model: a pytorch neural network model (takes x to log probability)
        x:     a single image, or adversarial example
        target_class: the class label of x (as an integer)
        returns: tensor of fitnesses
        This source code is inspired by:
        https://github.com/nesl/adversarial_genattack/blob/2304cdc2a49d2c16b9f43821ad8a29d664f334d1/genattack_tf2.py#L39
    """
    model.eval()
    with torch.no_grad():
        log_probs = model(batch)
        s = 1 - torch.exp(log_probs[:, target]) # Sum of probabilities, minus target class
        f = log_probs[:, target_class] - torch.log(s)
        if batch.size()[0] == 1:
            return f.item()
        return f.flatten()

In [0]:
def selection(fitness_fun, population, model, data, target, idx, num_elite = 2, pop_size = 10):
  """
  Input
  fitness_fun: the objective function by which we're using to calculate the
               the fitness - in this case it's the loss function
  population: the population of individuals of which to select the number for
              the next generation
  model: the PyTorch NN model 
  data: the input value to find a perturbation for
  target: the actual class of the input value
  idx: the generation number we're on - for debugging purposes
  num_elite: the number of elites to carry on from the previous generations
  pop_size: the size of each population
  Output
  new_pop: Returns the new population of individuals
  """
  #Find fitness for every individual in the population
  fitness = []
  for ind in population:
    #Take original data and add pertubation from GA individual
    x = data + ind
    data_fit = fitness_fun(model, x, target)
    #Append the perturbed data and the fitness value to the list
    fitness.append(ind, data_fit)

  #List of individual and their proportional fitness
  tot_fit = sum(x[1] for x in fitness)
  prob =  [x[1]/tot_fit for x in fitness]
  data_ind = [x[0] for x in fitness]
  fit = list(zip(data_ind, prob))
  #Sort the list by decreasing order - largest fitness first
  fit.sort(key=lambda x: x[1], reverse=True)

  #Initialize new population by adding the elites
  new_pop = fit[:num_elite]
  #Use this line if we only want to add the data, no fitness
  new_pop = [x[0] for x in new_pop] 

  #Create roulette wheel to get parents
  a = min(fit)[1] #min bound 
  b = max(fit)[1] #max bound
  i = 0
  parents = []
  fit_parent = []

  #Need 2*(pop_size - num_elite) parents
  #Already n elites, the rest are children and need two parents for one child
  while i < 2*(pop_size-num_elite): #Fill parents until it's the desired size
    t = (b-a)*np.random.rand(1) + a  #The value to see if the current parent is chosen
    j = random.choice(fit) #Picks a random tuple from the list
  
    if j[1] > t: #If fitness is greater than t add to parents - This is what assures that less fit solutions are less likely to be picked
      parents.append(j[0]) #Pick parent - only append data (can change if we do want fitness)
      fit_parent.append(j[1]) #Append the appropriate fitness as well to use
      i += 1

    #Will have duplicates, but an individual can mate more than once
  k = 0
  child_pop = []
  while k < pop_size:
    #Get two parents next to each other and their fitness then get their child
    p1 = parents[k]
    f_p1 = fit_parent[k]
    p2 = parents[k+1]
    f_p2 = fit_parent[k+1]
    child_pop.append(crossover(p1, f_p1, p2, f_p2))
    k += 2
    
  #Take the new children and mutate them - add mutated children to new_pop
  new_pop.extend(mutation_op(child_pop, data, idx))
  
  return new_pop


In [0]:
# ll = []
# for i in range(10):
#   ll.append((i, i*2))
# print(random.choice(ll))

(6, 12)


In [0]:
# fitness = []
# for i in range(10):
#   fitness.append((i, i+2))

# totfit = sum(x[1] for x in fitness)
# prob =  [x[1]/totfit for x in fitness]
# data = [x[0] for x in fitness]
# fit = list(zip(data, prob))
# fit.sort(key=lambda x: x[1], reverse=True)


In [0]:
# j = random.choice(fit)
# print(j)

(0, 0.03076923076923077)


In [0]:
# def crossover(parent1, parent2):
#   """
#   Point selection cross-over
#   From 0-num is parent 1, from num-end is parent 2
#   Implemented so num can never be 0 or end
#   input
#   parent1: individual in old population
#   parent2: individual in old population
#   output
#   child: new individual from mating of parets
#   """
#   n = len(parent1) - 1
#   cx = np.random.randint(1, n-1) #choose a random crossover point
#   child = parent1[:cx]+parent2[cx:]
#   return child

In [0]:
def crossover(parent1, parent2, p1, p2):
    """
    Point selection cross-over
    From 0-num is parent 1, from num-end is parent 2
    Implemented so num can never be 0 or end
    input
    parent1: individual in old population
    parent2: individual in old population
    output
    child: new individual from mating of parents
    """
    p = p1/(p1 + p2)
    mask = torch.empty_like(parent1).bernoulli_(p)
    child = mask * parent1 + (1 - mask) * parent2
    return child